# 항목 메타데이터 검증 및 가져오기 <a class="anchor" id="top"></a>

이 노트북에서는 `01_Validating_and_Importing_User_Item_Interaction_Data.ipynb`에 이어서 작업 항목 메타데이터 데이터 세트를 만듭니다. 그러면 필터를 사용하여 작업하고 나중에 `User Personalization` 또는 `HRNN-Metadata` 알고리즘을 지원할 수 있습니다.


이 노트북을 실행하려면, 데이터 세트를 만들고 상호 작용 데이터를 Amazon Personalize로 가져왔던 이전 노트북 `01_Validating_and_Importing_User_Item_Interaction_Data`를 실행해야 합니다. 이 노트북의 마지막에 변수 값 중 일부를 저장했습니다. 이제 이 값을 이 노트북에 로드해야 합니다.

In [ ]:
%store -r

## 항목 메타데이터 준비 <a class="anchor" id="prepare"></a>
[맨 위로 이동](#top)

다음으로 데이터를 로드하고 데이터가 양호한 상태인지 확인한 다음, 데이터를 CSV에 저장하여 Amazon Personalize에 사용할 준비가 되도록 합니다.

시작하기 위해 데이터 과학에서 일반적으로 사용되는 Python 라이브러리 컬렉션을 가져옵니다.

In [ ]:
import time
from time import sleep
import json
from datetime import datetime
import boto3
import pandas as pd

그런 다음 데이터 파일을 열고 처음 몇 행을 살펴봅니다.

In [ ]:
original_data = pd.read_csv(dataset_dir + '/movies.csv')
original_data.head(5)

In [ ]:
original_data.describe()

여기서는 데이터 세트에 대해 자세히 알 수 없으므로, 개략적인 정보를 얻기 위해 조금 더 살펴보겠습니다. 주로 장르별로 그룹화된다는 것을 알 수 있으며, Personalize가 이 구조를 지원하므로 괜찮은 형태입니다.

In [ ]:
original_data.info()

이와 관련해, 데이터 세트에 3개의 열에 걸쳐 총 (소규모의 경우 전체 9,742개에 대해 62,000개 이상)개의 항목이 있음을 알 수 있습니다.

이는 각 항목에 적용할 수 있는 movieId, 제목 및 장르 목록만 포함하는 최소한의 데이터 세트입니다. 하지만 Movielens 데이터 세트에는 추가 데이터가 있습니다. 예를 들어 제목에는 영화 개봉 연도가 포함됩니다. 다른 메타데이터 열을 만들어 보겠습니다.

In [ ]:
original_data['year'] =original_data['title'].str.extract('.*\((.*)\).*',expand = False)
original_data.head(5)

항목 메타데이터의 관점에서, 모델 훈련 및/또는 필터링 결과와 관련된 정보만 포함하려고 하므로 제목을 삭제하고 장르 정보는 유지합니다.

In [ ]:
itemmetadata_df = original_data.copy()
itemmetadata_df = itemmetadata_df[['movieId', 'genres', 'year']]
itemmetadata_df.head()

데이터를 조작한 후에는 항상 데이터 형식이 변경되었는지 확인합니다.

In [ ]:
itemmetadata_df.dtypes

Amazon Personalize에는 `movieId`에 매핑되는 `ITEM_ID`에 대한 기본 열이 있습니다. 이제 `GENRE`도 지정하면 더 많은 정보를 얻을 수 있습니다.

In [ ]:
itemmetadata_df.rename(columns = {'genres':'GENRE', 'movieId':'ITEM_ID', 'year':'YEAR'}, inplace = True) 

이제 끝났습니다. 이제 데이터가 사용 가능한 상태로 준비되었으므로, CSV 파일로 저장하기만 하면 됩니다.

In [ ]:
itemmetadata_filename = "item-meta.csv"
itemmetadata_df.to_csv((data_dir+"/"+itemmetadata_filename), index=False, float_format='%.0f')

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### 데이터 세트 생성

먼저 업로드하려는 데이터 세트의 유형을 Amazon Personalize에 알려주는 스키마를 정의합니다. 스키마에는 데이터 세트 유형에 따라 몇 가지 예약 키워드와 필수 키워드가 필요합니다. 자세한 내용은 [설명서](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html)를 참조하세요.

여기서는 `ITEM_ID` 및 `GENRE` 필드가 필요한 항목 메타데이터 데이터에 대한 스키마를 만듭니다. 데이터 세트에서 나타나는 순서와 동일한 순서로 스키마에 정의해야 합니다.

In [ ]:
itemmetadata_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "GENRE",
            "type": "string",
            "categorical": True
        },{
            "name": "YEAR",
            "type": "int",
        },
        
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-poc-movielens-item",
    schema = json.dumps(itemmetadata_schema)
)

itemmetadataschema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

생성된 스키마를 사용하여 데이터 세트 그룹 내에 데이터 세트를 만들 수 있습니다. 데이터는 아직 로드하지 않는다는 점을 유의하세요. 데이터는 몇 단계 후에 로드합니다.

In [ ]:
dataset_type = "ITEMS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-poc-movielens-items",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = itemmetadataschema_arn
)

items_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

### S3에 데이터 업로드

이제 Amazon S3 버킷이 생성되었으므로 사용자-항목-상호 작용 데이터의 CSV 파일을 업로드합니다. 

In [ ]:
itemmetadata_file_path = data_dir + "/" + itemmetadata_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(itemmetadata_filename).upload_file(itemmetadata_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+itemmetadata_filename

## 항목 메타데이터 가져오기 <a class="anchor" id="import"></a>
[맨 위로 이동](#top)

앞서 데이터 세트 그룹과 데이터 세트를 만들어 정보를 저장했으므로, 이제 S3 버킷에서 Amazon Personalize 데이터 세트로 데이터를 로드하는 가져오기 작업을 실행합니다. 

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-poc-item-import1",
    datasetArn = items_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, itemmetadata_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

데이터 세트를 사용하려면 먼저 가져오기 작업이 활성 상태여야 합니다. 아래의 셀을 실행하고 활성 상태로 표시될 때까지 기다립니다. 이 셀은 최대 6시간 동안 매초마다 가져오기 작업의 상태를 확인합니다.

데이터 세트의 크기에 따라 데이터를 가져오는 데 시간이 걸릴 수 있습니다. 이 워크숍에서는 데이터 가져오기 작업에 약 15분이 소요됩니다.

In [ ]:
%%time

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

이제 가져오기가 완료되면 추천에 대한 필터링과 `HRNN-Metadata` 지원을 활성화할 수 있습니다. 다음 셀을 실행하여 다음 노트북에 사용할 몇 가지 값을 저장하세요. 셀을 완료한 후 계속하려면 `03_Creating_and_Evaluating_Solutions.ipynb` 노트북을 여세요.

In [ ]:
%store items_dataset_arn
%store itemmetadataschema_arn